In [ ]:
import re
import tensorflow.keras as Keras

Tokenizer =  Keras.preprocessing.text.Tokenizer

filename = 'drive/My Drive/recurrentnetdataset.txt'

with open(filename, encoding='utf-8-sig') as f:
  text = f.read()

In [ ]:
seq_length = 20
start_story = '|'*seq_length

In [ ]:
text = text.lower()
text = start_story + text
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

In [ ]:
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
token_list = tokenizer.texts_to_sequences([text])[0]

In [ ]:
import numpy as np
np_utils = Keras.utils

In [ ]:
def generate_sequences(token_list, step):
  x = []
  y = []
  for i in range(0, len(token_list)-seq_length, step):
    x.append(token_list[i: i+seq_length])
    y.append(token_list[i+seq_length])

  y = np_utils.to_categorical(y, num_classes=total_words)

  num_seq = len(x)

  print('Number of sequneces: ', num_seq, '\n')
  return x ,y, num_seq

In [ ]:
step = 1
seq_length = 20
x, y, num_seq = generate_sequences(token_list, step)

x_train = np.array(x)
y = np.array(y)

Number of sequneces:  58911 



In [ ]:
Dense = Keras.layers.Dense
LSTM = Keras.layers.LSTM
Input = Keras.Input
Embedding = Keras.layers.Embedding
Dropout = Keras.layers.Dropout
Model = Keras.models.Model

In [ ]:
n_units = 256
embedding_size = 100

text_in = Input(shape=(None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation='softmax')(x)

In [ ]:
model = Model(text_in, text_out)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         488100    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 4881)              1254417   
Total params: 2,108,085
Trainable params: 2,108,085
Non-trainable params: 0
_________________________________________________________________


In [ ]:
RMSprop = Keras.optimizers.RMSprop
opti = RMSprop(0.001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opti)

In [ ]:
epochs = 100
batch_size = 32
model.fit(x = x_train, y =y, epochs = epochs, batch_size=batch_size, validation_data=None)

Epoch 1/100
1841/1841 [==============================] - 13s 7ms/step - loss: 5.2197
Epoch 2/100
1841/1841 [==============================] - 13s 7ms/step - loss: 4.8286
Epoch 3/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.6830
Epoch 4/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.5844
Epoch 5/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.5280
Epoch 6/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.5069
Epoch 7/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.4940
Epoch 8/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.5641
Epoch 9/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.6770
Epoch 10/100
1841/1841 [==============================] - 12s 7ms/step - loss: 4.8058
Epoch 11/100
1841/1841 [==============================] - 13s 7ms/step - loss: 4.9350
Epoch 12/100
1841/1841 [==============================] - 12s 7

In [ ]:
def sample_with_temp(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

def generate_text(seed_text, next_words, model, max_sequence_len, temp):
  output_text = seed_text
  seed_text = start_story + seed_text
  print(seed_text)

  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = token_list[-max_sequence_len:]
    token_list = np.reshape(token_list, (1, max_sequence_len))

    probs = model.predict(token_list, verbose=0)[0]
    y_class = sample_with_temp(probs, temperature=temp)
    output_word = tokenizer.index_word[y_class] if y_class >0 else ''

    if output_word == '|':
      break
    seed_text +=  output_word + ' '
    output_text += ' ' + output_word

  return output_text

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.3)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so all about before about the like about before about before came to all about one it all about before about one it all at all came to so all about before about one it there so all at came to out one it there that by one it all about to without found along so all again , out one it there at before about before about the like about one it all at all about before about put just at the like came to all about one it all too all about before came to so all as well , out all came to so all about one it all about one it all too all at all about before about one it all about one it all about one it all the like came to without found along so all at all at caught no one it all at all about one it there it all at the like came to no one it all at all about one it all at all about before about one it all at caught only without found along so well , without found along so well , out all at all at came to so about all about one it all about one it all at all before about one 

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.4)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so without found out one it all before about after came to so how about before about one it all about one it all too all came to so all at turned as got the that by that by so well , out all the like came to all again , out all too all about just at all before came to so all too all at set don\'t at got the like came to so without found about before about after came to so all without found along so without found along so before about the that by the that , at came to so well , out all at all along so all without which at set : without at got all as well , out one it all too all about the like about the like came to without found about before about before about fell what up one it all too all at so all and on turned as well , out it all again , out all too all as well , all again , out all too all about to all the like out all again , out all came to all the like came " all at heard the like turned the like a , without found about before came to turned the like ca

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.5)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so from turned the like came to without found along too all about one it all again , without found about all at the like about the one it so all as turned the like out one it all as well , out there some one it there that on turned as put just on turned the like turned for at all too all at before about set like a like came to without found about before about just at all again , without that on so than about one it all about one it all came to so well , without found about got a like turned as well , out one it all came to at heard the like about the than out all at all the like came to turned the like a than out all about the like along so well , without the like turned for about all from turned to so without found did came so all about before about before about to so all about before out all too all at came to so all about just as well , out some one it all came to so no one it all at came to all before about all at all came only without found out an like heard

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.55)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so all about the like about one it all as again , out it all about to all came to all before about before about came to so this she for out all about one it there it all at turned at caught no one there that at so this no one it all too all about again , to just at caught no like along so all about after came to turned as got the without found about to so all on the like about before about one it all again , without which at came to so on the like fell to without out all without found about the by that on turned the like about without the at set have no like came to so all too all about before came and at without found about one it so well , out all about again your too all at got the one there a like came to so all came not so without found thought he so from came no like turned at caught there so well , out one it all came the at all at only by just at like turned the like a like found along when the like under found along so well , without at turned as found a

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.6)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  no one it all at turned the like about caught do without came to so well , out one it there my like came to without which in well , to all came to all at turned for on looked all as again all turned the like turned the once as well all the , off there some one it all found along so men all came to so all at found went it there that on turned off so all came to so all without at caught only from turned his that by well , than from turned the like about one it that on did came have at caught no like heard the than out all at all and at all at all about to all about the like out my like came to all before about without that , without a than about too all at asked if from turned at like set what out so all as well , out one it all again i one it all without on went there so before found thought came to no one it all too all came there there that by as well , off so well , without found would like turned as got the by that by the of at got the like came to so all at w

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.65)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  there it all found about came to which by all about before out all about one it so before at caught no one it so among got one it all at all too all turned the so without which there there there it so than at turned at all as well all by so all to at the like turned from turned to without came to which at turned for on turned for came to without found along so than the like ran for out one it all came to so all too all too all about one it there that from turned for on turned out all about just at set to so all about after came to so that by so what up one it that at so like along better for together you all came to so from got the like about again , out some if out some one it all before turned off there so without out all about just at like turned at got the like came to so before thought the of so how about before the like at out all at all came only without at found out it all about before about one it all on without found about one it there some at set : sho

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.7)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  no before about put just like out one it all at caught only again , from got all about again , in just at set have lost came to no again . will like up only without found along the like about again , out when , from came to out the so about turned the like fell a too all too all along so about before turned when the than out all about put just from found dog she ; without on found along that , over went there so without found did came if into the like about to so from got the like a like fell if on out all out so without that on too all who so without a than along so men , on along not when that by got as well all again i all as set when , from turned the as well all about caught only without a has about after fell there an like at fell it only in only without found along now , one there all too all without found heard the like came be before about one so before found went what out there some one it all before about before about well your too all at fell what out

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.75)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so all just on had before about after fell if on the like a like before about there so all at out there a well , on well , out some what had as only on before too all along so as the by that by that on caught no before out one it that on as well , from turned to no before may at came you all made where along so well , without like turned as well , out all without which at even caught how along when the like under came to all into turned no one there it no all down got the than up the like which for about set : should at all before again , without on too all one it no like about after turned before all about to so without on got a too if at came to all at all at came to without found about all at caught only about dogs all into the than out one it all about after came to so without heard the to without that about again , out all too like came to turned what came so from turned like about one there that on after found got as all about before about well , without he

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.8)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  no so this she at all on just at all so that by one it all the like found along good without found thought all as again , on just away at got the like about one it all out it all out he what he no one it all that by that on too all about again , from turned for into the like got as thought himself all the than upon place every as well , to without at set to all at so well , without found along the like by my better when the to found went it there some to so all before saw dog him there some asked if on got then , without came the like turned out one that out all into the like a too my one there that on just at one it no like about found about the very you all the like out some one it there so men that on did came every on caught you all well all too all at than from turned by the like a again , on the ? off there when , in the that by just that by just out so from turned for first found when the like down one it no like along so at one it all the as again there s

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 0.9)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  not there that , off one it all what under caught not so men well , and on got the of at turned the so home before caught if along so than about the like down went no there that on fell my king not a too all at all ; without long there so all again . set have all and on before that on went there , no like one it once this so well , for at came not all who them all at without found about set not all too it all at without found about one it there at up not so any at caught only without which upon man there no before put said jupiter there so all as well an at all again , out you all looked all came to turned at all the like , on without at first one an , a like before about means my has got the as about again , out so under thought one it no like turned the for thought he so about who the like came to all before too my very will all round at like went what by so all too all again you all into the of at caught got the like about one it so without heard the like with

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 1.3)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  so without came you one she at before about before never at turned " up how up put said he there in had when along for up one it so out an one it that by when of it so without off it all when , you all found again his why , that on without at so this so he so - all go one there some the for away that , without at before up so all another all too what a like about one that , from put so set when of at caught what found out one it all from now in all among came there some they what up if came to like off this then the , all about as again there an that is from off there an more it she : can at more a not so before replied then the just it well off all found about men that in him all turned it jupiter there he just and at have these , from got the in too my , it only without turned before off so without then by it a just that over put your , too my without found together we than turned without the than into the like upon sheep as set to the at got all at come little

In [ ]:
generate_text('the ass and his lion ', 500, model, 1, 1.5)

||||||||||||||||||||the ass and his lion 


'the ass and his lion  no means his so off i take with had at much back if down so without away that about all the no her make too him he all came let much got the more a of up were one it all took dogs no of at like to so just from came " them so than off she my ! came there so too bull . your just turned there your and on first who i there my , without about among just from again there as home before ran at at heard would now for into turned off who all after without found out if up one all before must like master if a and into turned at heard before in so this there like over then - so of caught then as found along then just it which at got as turned as ran it all after found went you one so no home , it his like before he so they , he what out replied one some because up work , that , of all first put than down he so that there so this so nothing when : should one it my better no what fell what at again that which at all lay an than on i\'m - what a of got now well that about well 